### House Price Prediction Exercise - data_description.txt
SalePrice — the property’s sale price in dollars. This is the target variable that you’re trying to predict.  
MSSubClass — The building class  
MSZoning — The general zoning classification 
MasVnrArea - Masonary Veneer Area in sq ft
LotFrontage — Linear feet of street connected to property  
LotArea — Lot size in square feet  
Street — Type of road access  
Alley — Type of alley access  
LotShape — General shape of property  
LandContour — Flatness of the property  
Utilities — Type of utilities available  
LotConfig — Lot configuration  

In [ ]:
# House Sales Data, ETL, Build Predictive Model(linear regression), 
# Assess Model, Deploy Model. Training data  'house100Data.csv'
# R^2 = 0.75  Price explained by the indep. variables
import Pkg; Pkg.add("DataFrames")
Pkg.add("Missings")
Pkg.add("Plots")
Pkg.add("CSV")
using DataFrames
using CSV
using Missings
using Plots

$$Y = X\beta + e $$

In [ ]:
using DataFrames; 
data = CSV.read("house100Data.csv", DataFrame)
train = data[1:100,:]
size(train)

In [ ]:
# extract SalePrice column
price = train[1:3, :SalePrice]




In [ ]:
# plot on scatter plot
using Plots
# plot SalePrice vs id
#plot([1:100], price)
# scatter plot
scatter(train[:,:LotArea], price)
# bar plot
#bar([1:100], price)


In [ ]:
# drop columns with missing values
# drop columns with missing values
#select!(train, Not(:Alley))   # this drops the Alley column
#shape of train
size(train)

In [ ]:
df2 = DataFrame(:A => [5, 10, 15, 20, 25], :Y => [5, 10, missing, 20, 15])

In [ ]:
# count missing values in column 
function getListOfMissingColumns(df)
    missingColumns = []
    for i in 1:size(df)[2]
        num = count(ismissing, df[:,i])
        if num > 0
            # print column name and number of missing values
            println(names(df)[i], " ", num)
            push!(missingColumns, names(df)[i])
        end
    end
    return missingColumns
end
getListOfMissingColumns(train)
# drop columns with missing values




In [ ]:
function dropColumnsWithMissingValues(df)
    missingColumns = getListOfMissingColumns(df)
    for i in 1:length(missingColumns)
        select!(df, Not(Symbol(missingColumns[i])))
    end
    return df
end
train = dropColumnsWithMissingValues(train)
size(train)

In [ ]:
# get names of non numeric columns float and int

function getNonNumericColumns(df)
    nonNumericColumns = []
    for i in 1:size(df)[2]
        if !(eltype(df[:,i]) <: Number)
            push!(nonNumericColumns, names(df)[i])
        end
    end
    return nonNumericColumns
end
nonNumericCols = getNonNumericColumns(train)


In [ ]:
# drop non numeric columns
function dropNonNumericColumns(df)
    nonNumericColumns = getNonNumericColumns(df)
    for i in 1:length(nonNumericColumns)
        select!(df, Not(Symbol(nonNumericColumns[i])))
    end
    return df
end
train = dropNonNumericColumns(train)
size(train)


In [ ]:
# get name of problem column as visbile in heatmap



In [ ]:
# Build correlation matrix
using Statistics
cor(Matrix(train))
# Plot heatmap of correlation matrix
heatmap(cor(Matrix(train)))

In [ ]:
# select top 4 columns with highest correlation with SalePrice

function getTopCorrelatedColumns(df, num)
    corMatrix = cor(Matrix(df))
    # get column names
    colNames = names(df)
    # get correlation values
    corValues = corMatrix[:,size(df)[2]]
    # sort correlation values
    sortedCorValues = sort(corValues, rev=true)
    # get top 5 values
    topCorValues = sortedCorValues[1:num]
    # get column names of top 5 values
    topCorColumns = []
    for i in 1:num
        for j in 1:size(df)[2]
            if corValues[j] == topCorValues[i]
                push!(topCorColumns, colNames[j])
            end
        end
    end
    return topCorColumns
end
topCorColumns = getTopCorrelatedColumns(train, 5)

In [ ]:
size(train)

In [ ]:
# select columns from list of column names
function selectColumns(df, colNames)
    X = []
    for i in 1:length(colNames)
        if i == 1
            X = df[:,Symbol(colNames[i])]
        else
            X = hcat(X, df[:,Symbol(colNames[i])])
        end
    end
    return X
end


Y = selectColumns(train, topCorColumns[1:1])
X = selectColumns(train, topCorColumns[2:4])

In [ ]:
# build linear regression model
using Statistics
using LinearAlgebra
function buildLinearRegressionModel(X, Y)
    # get number of rows
    m = size(X)[1]
    # add column of ones to X
    X = hcat(ones(m,1), X)
    # get theta
    theta = inv(X' * X) * X' * Y
    return theta
end
model = buildLinearRegressionModel(X, Y)



In [ ]:
# predict price
function predictPrice(model, X)
    # get number of rows
    m = size(X)[1]
    # add column of ones to X
    X = hcat(ones(m,1), X)
    # get theta
    Y = X * model
    return Y
end
Yp = predictPrice(model, X)
# model accuracy R^2



In [ ]:
# plot predicted price vs actual price
scatter(Y, Yp, label="Predicted vs Actual Price")

In [ ]:
# RMS difference between predicted and actual price
function getRMS(Y, Yp)
    # get number of rows
    m = size(Y)[1]
    # get difference
    diff = Y - Yp
    # get square of difference
    diffSq = diff .^ 2
    # get sum of square of difference
    sumDiffSq = sum(diffSq)
    # get mean of square of difference
    meanDiffSq = sumDiffSq / m
    # get root of mean of square of difference
    rms = sqrt(meanDiffSq)
    return rms
end
rms = getRMS(Y, Yp)




In [ ]:
# get R^2 value. Metric to measure accuracy of model
function getR2Accuracy(Y, Yp)
    # get number of rows
    m = size(Y)[1]
    # get mean of Y
    Ymean = mean(Y)
    # get sum of squared errors
    SSE = sum((Y - Yp).^2)
    # get sum of squared total
    SST = sum((Ymean .-Y).^2)  # note the dot operator and order of subtraction
    # get R^2
    R2 = 1 - SSE/SST
    return R2
end 
getR2Accuracy(Y, Yp)


In [ ]:
# selet numeric columns only
numericCols = [:LotArea, :OverallQual, :OverallCond, :YearBuilt, :YearRemodAdd, :MasVnrArea, :BsmtFinSF1, :BsmtFinSF2, :BsmtUnfSF, :TotalBsmtSF, :1stFlrSF, :2ndFlrSF, :LowQualFinSF, :GrLivArea, :BsmtFullBath, :BsmtHalfBath, :FullBath, :HalfBath, :BedroomAbvGr, :KitchenAbvGr, :TotRmsAbvGrd, :Fireplaces, :GarageYrBlt, :GarageCars, :GarageArea, :WoodDeckSF, :OpenPorchSF, :EnclosedPorch, :3SsnPorch, :ScreenPorch, :PoolArea, :MiscVal, :MoSold, :YrSold]


In [ ]:
# plot histogram of predicted price
using Plots
histogram(Yp, label="Predicted Price")


In [ ]:
# Problem Set
# 1. Build a linear regression model to predict the price of houses in the test.csv file
# 2. Submit your predictions 


test = CSV.read("test.csv", DataFrame)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.pairplot(numeric[cols], height=2.5)
plt.show()

In [ ]:
# usefule functions 
names(train)[33]
sum(train[:,33] )
# drop problem column
select!(train, Not(:PoolArea))
size(train)